This notebook pulls 2010 census data and attempts to generate a large set of points that approximates a smooth surface

In [1]:
# Declare static variables

n=50 # The number of points to assign to each census block

In [2]:
# Import libraries

import pandas as pd
import geopandas
import numpy as np
import requests
from io import BytesIO
import folium
from IPython.display import clear_output

In [3]:
# Request shapefile data for 2010 census tracts and convert to geopandas dataframe

# Shapefile url
data_url = 'https://www2.census.gov/geo/tiger/GENZ2010/gz_2010_10_140_00_500k.zip'


# Request data
data = requests.get(data_url)
# convert to pandas dataframe
tract_data = geopandas.read_file(BytesIO(data.content))

In [4]:
# Request shapefile data for 2010 census tracts and convert to geopandas dataframe

# Shapefile url
data_url = 'https://www2.census.gov/geo/tiger/TIGER2010/TABBLOCK/2010/tl_2010_10_tabblock10.zip'


# Request data
data = requests.get(data_url)
# convert to pandas dataframe
block_data = geopandas.read_file(BytesIO(data.content))

In [5]:
# For each census block, create a bounding box
block_bounds = block_data["geometry"].bounds

# Attch GEOID to boundaries
block_bounds = block_data[["GEOID10","geometry"]].merge(block_bounds, left_index=True, right_index=True)

In [6]:
# Fit a 2D Gaussian distribution over the bounding boxes

# Takes in a row of 'block_bounds' and outputs a 2D Gaussian distribution of 'n' points over the bounding box, as well as the GEOID
def get_points(row,n):
    print(f"Processing Block {row['GEOID10']}...")
    # 'i' is the total number of points left to assign
    i=n
    # 'points_return' is the list of all points for the block
    # TODO: CRS is hardcoded
    points_return = geopandas.GeoSeries(crs="EPSG:4269")
    # Allocate points until n have been assigned
    while i > 0:
        # Generates a Gaussian distribution for the y-axis, located at the center of the box with standard deviation such that 95% of the distribution lies in the bounding box
        pointsy = np.random.normal(loc=(row["maxy"] + row["miny"])/2, scale=np.abs(row["maxy"] - row["miny"])/1.96, size=i)
        # Generates a Gaussian distribution for the x-axis, located at the center of the box with standard deviation such that 95% of the distribution lies in the bounding box
        pointsx = np.random.normal(loc=(row["maxx"] + row["minx"])/2, scale=np.abs(row["maxx"] - row["minx"])/1.96, size=i)
        # Convert the points to Shapely points
        points = geopandas.GeoSeries(geopandas.points_from_xy(pointsx, pointsy, crs="EPSG:4269"))
        # Check if the points are inside the block
        point_checks = points.within(row["geometry"])
        # Add found points to our list
        points_return = geopandas.GeoSeries(pd.concat([points_return, points[point_checks]], ignore_index=True), crs=points_return.crs)
        # Set 'i' equal to the number of missed points
        i = n - points_return.size
    
    # Clear warnings from notebook output to prevent crash
    clear_output()
    # Return an array with every point in the cloud and the GEOID
    return np.append(points_return.values, row["GEOID10"])
    

In [7]:
# Fit a Gaussian distribution to each block
point_cloud = block_bounds.apply(get_points, axis=1, args=(n,), result_type='expand')

# Rename columns of the pointcloud
point_cloud.columns = ['GEOID' if str(x)==str(n) else 'point_' + str(x) for x in point_cloud.columns]

In [8]:
# Pull population data for 2010 Census blocks
# Define request parameters

year = '2010' # Year of interest
datasource = 'dec' # Survey name
subsource = 'pl' # Subsurvey name
GET = 'P001001,H001001,P001003' # Variables to query
FOR = 'block:*' # for predicate
IN = 'state:10&in=county:*&in=tract:*'

# Filepath to your Census API key
keyfile = 'CensusAPIKey.txt'

# Formatted API call
data_url = f'https://api.census.gov/data/{year}/{datasource}/{subsource}?get={GET}&for={FOR}&in={IN}'

# Read Census key into 'api_key'
with open(keyfile) as key:
    api_key = key.read().strip()

# Add key to url
data_url = f'{data_url}&key={api_key}'

# Request data and convert from json
data = requests.get(data_url).json()
# First entry in list is a list of variable names
data = pd.DataFrame(data[1:], columns = data[0])

# Rename columns to match shapefile pull
data.rename(columns = {"state":"STATEFP10", "county":"COUNTYFP10", "tract":"TRACTCE10", "block":"BLOCKCE10"}, inplace=True)

# Attach to block shapes
block_data = block_data.merge(data, on=["STATEFP10","COUNTYFP10","TRACTCE10","BLOCKCE10"])

In [9]:
# Pull population data for 2010 Census tracts
# Define request parameters

year = '2010' # Year of interest
datasource = 'dec' # Survey name
subsource = 'pl' # Subsurvey name
GET = 'P001001,H001001,P001003' # Variables to query
FOR = 'tract:*' # for predicate
IN = 'state:10' # in predicate


# Filepath to your Census API key
keyfile = 'CensusAPIKey.txt'

# Formatted API call
data_url = f'https://api.census.gov/data/{year}/{datasource}/{subsource}?get={GET}&for={FOR}&in={IN}'

# Read Census key into 'api_key'
with open(keyfile) as key:
    api_key = key.read().strip()

# Add key to url
data_url = f'{data_url}&key={api_key}'

# Request data and convert from json
data = requests.get(data_url).json()
# First entry in list is a list of variable names
data = pd.DataFrame(data[1:], columns = data[0])

# Rename columns to match shapefile pull
data.rename(columns = {"state":"STATE", "county":"COUNTY", "tract":"TRACT"}, inplace=True)

# Attach to tract shapes
tract_data = tract_data.merge(data, on=["STATE","COUNTY","TRACT"])

In [10]:
# Assign a fraction of the population of each block as a value to each point
point_cloud["population_per_point"] = block_data["P001001"].astype(int) / n

In [11]:
# Flatten to a GeoSeries where each row is a point and its weight
weights = np.array([[row["population_per_point"] for _ in range(n)] for i, row in point_cloud.iterrows()]).flatten()
points = np.array([[row["point_" + str(i)] for i in range(n)] for _, row in point_cloud.iterrows()]).flatten()
points_list = geopandas.GeoDataFrame({"population_per_point":weights,"geometry":points}, crs="EPSG:4269")

# Determine the number of points in the point cloud. This should be n * the number of census blocks
print(points_list.shape[0] / n == block_data.shape[0])

/Users/emiller/opt/anaconda3/envs/census-tract-redistricting/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3378: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  exec(code_obj, self.user_global_ns, self.user_ns)
/var/folders/9r/738fljg527g64ktptsnnzlpw0000gn/T/ipykernel_1361/4162940238.py:3: FutureWarning: The input object of type 'Point' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Point', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  points = np.array([[row["point_" + str(i)] for i in range(n)] for _, row in point_cloud.iterrows()]).flatten()
/var/folders/9r/738fljg52

True


In [ ]:
# Spatially join each point to the 2010 census tract containing it
variables_per_point = geopandas.sjoin(points_list, tract_data, how="left", op='within')

In [ ]:
# WARNING: Plot is large and should only be rendered if necessary
# TODO: Points around the edge of the state are being lost


# Find and plot all missed points 
missed_points = variables_per_point.loc[variables_per_point["index_right"].isna()]

# initialize the map and store it in a folium map object
us_map = folium.Map(location=[38.9108, -75.5277], zoom_start=8, tiles=None)

# Add background tiles
folium.TileLayer('CartoDB positron',name="Light Map",control=False).add_to(us_map)

# Style and highlight functions map population values to color values
style_function = lambda x: {"weight":0.5, 
                            'color':'black',
                            'fillColor':'red', 
                            'fillOpacity':0.75}

# Add a map over the tiles with the given colors and a tooltip
NIL=folium.features.GeoJson(
        missed_points, # Full geopandas data
        style_function=style_function, # function for base colors
        control=False
    )

# Add elements to map
us_map.add_child(NIL)

In [33]:
# Exclude missed points from the list
variables_per_point = variables_per_point.loc[~variables_per_point["index_right"].isna()]

# Divide variables of intersest by tract population and multiply by the portion of the population represented by each point
variables_per_point[["P001001", "H001001", "P001003"]] = variables_per_point[["P001001", "H001001", "P001003"]].astype(int).div(variables_per_point["P001001"].astype(int), axis=0).mul(variables_per_point["population_per_point"], axis=0)
# Reset index
variables_per_point = variables_per_point.reset_index()

/Users/emiller/opt/anaconda3/envs/census-tract-redistricting/lib/python3.9/site-packages/geopandas/geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


In [34]:
variables_per_point

,index,population_per_point,geometry,index_right,GEO_ID,STATE,COUNTY,TRACT,NAME,LSAD,CENSUSAREA,P001001,H001001,P001003
0,0,4.88,POINT (-75.48581 39.12138),87.0,1400000US10001041100,10,001,041100,411,Tract,6.190,4.88,1.49662,3.539957
1,1,4.88,POINT (-75.48533 39.12180),87.0,1400000US10001041100,10,001,041100,411,Tract,6.190,4.88,1.49662,3.539957
2,2,4.88,POINT (-75.48550 39.12075),87.0,1400000US10001041100,10,001,041100,411,Tract,6.190,4.88,1.49662,3.539957
3,3,4.88,POINT (-75.48712 39.12121),87.0,1400000US10001041100,10,001,041100,411,Tract,6.190,4.88,1.49662,3.539957
4,4,4.88,POINT (-75.48568 39.12272),87.0,1400000US10001041100,10,001,041100,411,Tract,6.190,4.88,1.49662,3.539957
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1198227,1205745,0.00,POINT (-75.46020 38.52854),176.0,1400000US10005051702,10,005,051702,517.02,Tract,68.279,0.00,0.00000,0.000000
1198228,1205746,0.00,POINT (-75.46595 38.52326),176.0,1400000US10005051702,10,005,051702,517.02,Tract,68.279,0.00,0.00000,0.000000
1198229,1205747,0.00,POINT (-75.45732 38.52271),176.0,1400000US10005051702,10,005,051702,517.02,Tract,68.279,0.00,0.00000,0.000000
1198230,1205748,0.00,POINT (-75.46772 38.53000),176.0,1400000US10005051702,10,005,051702,517.02,Tract,68.279,0.00,0.00000,0.000000


In [35]:
# Print the number of points missed in the transfer of data from tracts to points
print(points_list.shape[0] -  variables_per_point.shape[0])

7518


In [36]:
# Request shapefile data for 2020 census tracts and convert to geopandas dataframe

# Shapefile url
data_url = 'https://www2.census.gov/geo/tiger/GENZ2020/shp/cb_2020_10_tract_500k.zip'


# Request data
data = requests.get(data_url)
# convert to pandas dataframe
tract2020 = geopandas.read_file(BytesIO(data.content))

In [43]:
# Spatially join points to 2020 census tracts
interpolated_values = geopandas.sjoin(variables_per_point[["GEO_ID","geometry","P001001","H001001","P001003"]], tract2020, how="left", op='within')

# Sum the values for each 2020 tract
interpolated_values = interpolated_values[["GEOID", "P001001", "H001001", "P001003"]].groupby("GEOID").sum()

interpolated_values


#tract2020

,P001001,H001001,P001003
GEOID,,,
10001040100,6541.94,2469.421534,5703.400558
10001040201,5050.02,2026.237084,3671.148518
10001040203,4988.42,2006.704179,3104.394212
10001040204,4651.60,1731.582200,3158.091335
10001040205,2876.36,1070.837184,1952.589021
...,...,...,...
10005051702,5619.32,2305.549257,5018.622008
10005051801,4885.14,2117.297075,3854.988715
10005051802,4186.28,1753.492446,2500.699198


In [44]:
# Pull population data for 2020 Census tracts
# Define request parameters

year = '2020' # Year of interest
datasource = 'dec' # Survey name
subsource = 'pl' # Subsurvey name
GET = 'P1_001N,H1_001N,P1_003N' # Variables to query
FOR = 'tract:*' # for predicate
IN = 'state:10' # in predicate


# Filepath to your Census API key
keyfile = 'CensusAPIKey.txt'

# Formatted API call
data_url = f'https://api.census.gov/data/{year}/{datasource}/{subsource}?get={GET}&for={FOR}&in={IN}'

# Read Census key into 'api_key'
with open(keyfile) as key:
    api_key = key.read().strip()

# Add key to url
data_url = f'{data_url}&key={api_key}'

# Request data and convert from json
data = requests.get(data_url).json()
# First entry in list is a list of variable names
tract2020_data = pd.DataFrame(data[1:], columns = data[0])

# Add a GEOID column to the data
tract2020_data["GEOID"] = tract2020_data["state"].astype(str) + tract2020_data["county"].astype(str) +tract2020_data["tract"].astype(str)

In [45]:
# Write combined dataframe of 2020 ground truth and estimated values to a csv
interpolated_values.merge(tract2020_data, left_index=True, right_on="GEOID").to_csv("estimates.csv", index=False)

In [46]:
interpolated_values.merge(tract2020_data, left_index=True, right_on="GEOID")

,P001001,H001001,P001003,P1_001N,H1_001N,P1_003N,state,county,tract,GEOID
220,6541.94,2469.421534,5703.400558,7315,2740,5980,10,001,040100,10001040100
221,5050.02,2026.237084,3671.148518,5446,2123,3424,10,001,040201,10001040201
222,4988.42,2006.704179,3104.394212,5182,2157,2808,10,001,040203,10001040203
223,4651.60,1731.582200,3158.091335,6451,2269,3613,10,001,040204,10001040204
224,2876.36,1070.837184,1952.589021,4699,1985,2430,10,001,040205,10001040205
...,...,...,...,...,...,...,...,...,...,...
214,5619.32,2305.549257,5018.622008,6577,2590,5286,10,005,051702,10005051702
215,4885.14,2117.297075,3854.988715,5359,2154,3636,10,005,051801,10005051801
216,4186.28,1753.492446,2500.699198,4354,1740,2256,10,005,051802,10005051802
217,4556.50,1829.590777,3625.193469,4760,1949,3566,10,005,051900,10005051900
